In [1]:
%reset -sf

In [1]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
# import scipy
# import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [137]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip()

In [ ]:
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1

In [48]:
items = [[79, 98], [54, 65, 75, 74], [79, 60, 97], [74]]
ops = [
    lambda old: old * 19,
    lambda old: old + 6,
    lambda old: old * old,
    lambda old: old + 3,
]
test = [
    lambda x: x%23 == 0, lambda x: x%19 == 0, lambda x: x%13 == 0, lambda x: x%17 == 0, 
]
iftrue = [2,2,1,0]
iffalse = [3,0,3,1]
M = 4


In [65]:
items = [[77, 69, 76, 77, 50, 58], [75, 70, 82, 83, 96, 64, 62], [53], [85, 64, 93, 64, 99], 
         [61, 92, 71], [79, 73, 50, 90], [50, 89], [83, 56, 64, 58, 93, 91, 56, 65]]
ops = [
    lambda old: old * 11,
    lambda old: old + 8,
    lambda old: old * 3,
    lambda old: old + 4,
    lambda old: old * old,
    lambda old: old + 2,
    lambda old: old + 3,
    lambda old: old + 5,
]
test = [
    lambda x: x%5 == 0, lambda x: x%17 == 0, lambda x: x%2 == 0, lambda x: x%7 == 0, 
    lambda x: x%3 == 0, lambda x: x%11 == 0, lambda x: x%13 == 0, lambda x: x%19 == 0, 
]
iftrue = [1,5,0,7,2,4,4,1]
iffalse = [5,6,7,2,3,6,3,0]
M = 8

In [66]:
pdt = 5*17*2*7*3*11*13*19

In [67]:
cnts = [0 for _ in range(M)]
for _ in range(10_000):
    for midx in range(M):
        items[midx] = items[midx][::-1]
        while items[midx]:
            val = items[midx].pop()
            val = ops[midx](val)
#             val = val // 3
            val = val%pdt
            if test[midx](val):
                items[iftrue[midx]].append(val)
            else:
                items[iffalse[midx]].append(val)
            cnts[midx] += 1
#     print(items)

In [68]:
cnts

[115025, 93029, 114545, 114164, 8590, 30465, 122693, 122667]

In [69]:
sorted(cnts)[-1] * sorted(cnts)[-2]

15050382231

In [6]:
srr = """
""".strip()

In [138]:
srr = """
""".strip()

In [139]:
res = 0

In [15]:
def test(arr):
    res = 0
    return res